In [1]:
# 📦 Imports
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import os
import pandas as pd
import torch.nn as nn


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# 🧹 Load preprocessed data
df = pd.read_csv("data/archive/processed_styles.csv")


In [10]:
# 📂 Dataset Class
class FashionDataset(Dataset):
    def __init__(self, dataframe, image_dir, encoders, transform=None):
        self.df = dataframe
        self.image_dir = image_dir
        self.transform = transform
        self.encoders = encoders

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, f"{row['id']}.jpg")
        try:
            image = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            image = Image.new("RGB", (128, 128), (255, 255, 255))
        if self.transform:
            image = self.transform(image)

        labels = {
            'gender': self.encoders['gender'].transform([row['gender']])[0],
            'color': self.encoders['baseColour'].transform([row['baseColour']])[0],
            'season': self.encoders['usage'].transform([row['usage']])[0],
            'product': self.encoders['masterCategory'].transform([row['masterCategory']])[0],
        }
        return image, labels


In [4]:
# 🔧 Transforms and DataLoader
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])


In [11]:
# 🎯 Training Setup

from sklearn.preprocessing import LabelEncoder

# Fit label encoders for each categorical column
encoders = {}
for col in ['gender', 'baseColour', 'usage', 'masterCategory']:
    le = LabelEncoder()
    le.fit(df[col])
    encoders[col] = le

dataset = FashionDataset(df, "data/archive/images", encoders, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
# 🧠 Multi-Output Model
class MultiOutputModel(nn.Module):
    def __init__(self, num_genders, num_colors, num_seasons, num_products):
        super().__init__()
        base = models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(base.children())[:-1])
        self.dropout = nn.Dropout(0.3)
        self.gender = nn.Linear(512, num_genders)
        self.color = nn.Linear(512, num_colors)
        self.season = nn.Linear(512, num_seasons)
        self.product = nn.Linear(512, num_products)

    def forward(self, x):
        x = self.features(x).squeeze()
        x = self.dropout(x)
        return {
            'gender': self.gender(x),
            'color': self.color(x),
            'season': self.season(x),
            'product': self.product(x)
        }

model = MultiOutputModel(
    num_genders=len(encoders['gender'].classes_),
    num_colors=len(encoders['baseColour'].classes_),
    num_seasons=len(encoders['usage'].classes_),
    num_products=len(encoders['masterCategory'].classes_)
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [ ]:
# 🔁 Training Loop
for epoch in range(5):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        # Stack label lists into tensors and move to device
        targets = {k: torch.tensor(v, dtype=torch.long, device=device) for k, v in labels.items()}
        outputs = model(images)

        # Compute loss for each output
        loss = sum([criterion(outputs[k], targets[k]) for k in outputs])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")


/var/folders/4x/l6k3_0sn1gjfk4pc969hrd9m0000gn/T/ipykernel_77034/3497749346.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = {k: torch.tensor(v, dtype=torch.long, device=device) for k, v in labels.items()}
/var/folders/4x/l6k3_0sn1gjfk4pc969hrd9m0000gn/T/ipykernel_77034/3497749346.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = {k: torch.tensor(v, dtype=torch.long, device=device) for k, v in labels.items()}
/var/folders/4x/l6k3_0sn1gjfk4pc969hrd9m0000gn/T/ipykernel_77034/3497749346.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.te

KeyboardInterrupt: 

In [ ]:
# 💾 Save Model
torch.save(model.state_dict(), "fashion_model.pth")
